# Migration matrix demo
This notebook demonstrates how to use the *crv_pylib* to create a migration matrix for the following input demo data:

* **PD FIRB data**. Resulting migration matrix: `mm`
* **Syntetic data**. Resulting migration matrix: `mm2`

Please refer to the documentation of the *crv_pylib* for details on the applied functions: */crv_pylib/docs/html/index.html*

#### Load libraries & set options

In [73]:
# imports and options
import numpy as np
import pandas as pd
from crv.validation.summary import migration_matrix
from crv.utils.dataframe_helper import rating_cats, scoring_cats_list, categorize
from data.rating_scoring_pds import sco_pd_dict
print(sco_pd_dict)

from IPython.display import display
pd.options.display.max_columns = 40

{'A+': 0.0008, 'A': 0.00114, 'A-': 0.00158, 'B+': 0.0022, 'B': 0.00308, 'B-': 0.00429, 'C+': 0.00597, 'C': 0.00837, 'C-': 0.01169, 'D+': 0.01638, 'D': 0.02297, 'D-': 0.03196, 'E+': 0.04472, 'E': 0.063, 'E-': 0.08792, 'F+': 0.12279, 'F': 0.1719, 'F-': 0.27028, '0+': 1, '0': 1, '0-': 1, 'U': 0.025, 'M': 0.025, 'N': 0.025, 'S': 0.025, '-': 0.025}


## PD FIRB data demo

### Step 1. Load & transform data
Note: Change path to point to folder containing demo data (i.e. path = "C:\\Users\\<<*your_user_id*>>\\Nordea\\CRV_PYLIB - Documents\\Testing Notebooks\\test_data\\").

In [74]:
# Sample data from SAPDDP
# years 2016 and 2017 that has been pd.pickled
# data setup
user_id = "G85544"
dfirb = pd.read_pickle("C:\\Users\\" + user_id + "\\Nordea\\CRV_PYLIB - Documents\\Testing Notebooks\\test_data\\dfirb_2016_2017.pkl")
## make a default column
dfirb['df_2'] = dfirb.FRATINGO.apply(lambda rating: 1 if '0' in rating else 0)
dfobs = dfirb[dfirb.CALMONTH=='201612']
dfperf = dfirb[dfirb.CALMONTH=='201712']

df1 = dfobs[['group', 'rating', 'FGRAD_MET', 'df_2']].copy()
df1 = df1.sort_values(by=['group', 'rating']).groupby('group').head(1)
##df1 = df1[~df1.rating.str.contains('0')]  # uncomment to exlcude zero ratings
df2 = dfperf[['group', 'rating', 'FGRAD_MET', 'df_2']].copy()
df2 = df2.sort_values(by=['group', 'rating']).groupby('group').head(1)
##df2 = df2[~df2.rating.str.contains('0')]  # uncomment to exclude zero ratings


### Step 2. Create the migration matrix

The before data, `data_0`, and the after data, `data_1`, are accepted as inputs, as well as the required columns. The function assumes that the default flag is in `data_0`.

In [75]:
mm = migration_matrix(data_0=df1, data_1=df2, 
                      id_col='group', rating_col='rating', 
                      model_col='FGRAD_MET', default_col='df_2')

print("Demo: Migration Matrix")
display(mm)

Demo: Migration Matrix


6+         6        6-        5+         5        5-        4+  \
6+  0.797161  0.080426  0.008279  0.002365  0.000591  0.000591  0.010645   
6   0.150769  0.600769  0.100769  0.026154  0.009231  0.003846  0.010769   
6-  0.022562  0.129549  0.533479  0.086608  0.024745  0.010189  0.026201   
5+  0.007683  0.024823  0.121158  0.408983  0.118203  0.031324  0.017139   
5   0.000368  0.004779  0.018750  0.110294  0.409191  0.096691  0.030882   
5-  0.001632  0.001305  0.008159  0.031658  0.138708  0.352154  0.109334   
4+  0.000000  0.000572  0.002002  0.008293  0.032313  0.126108  0.324278   
4   0.000741  0.000000  0.000741  0.003460  0.014829  0.042264  0.131735   
4-  0.001221  0.000000  0.001221  0.002746  0.007324  0.023192  0.056454   
3+  0.000775  0.000000  0.000775  0.003490  0.008143  0.014347  0.025979   
3   0.000000  0.000000  0.001040  0.003640  0.002600  0.006760  0.020801   
3-  0.000000  0.000605  0.001209  0.004837  0.003023  0.007860  0.011487   
2+  0.000000  0.000000  0.000000  0.000000  0.001376  0.002751  0.005502   
2   0.000000  0.000000  0.002137  0.000000  0.002137  0.002137  0.006410   
2-  0.002865  0.000000  0.000000  0.000000  0.000000  0.000000  0.002865   
1+  0.004854  0.000000  0.000000  0.000000  0.000000  0.000000  0.014563   
1   0.000000  0.000000  0.000000  0.005714  0.005714  0.005714  0.005714   
1-  0.000000  0.000000  0.000000  0.006579  0.000000  0.000000  0.006579   
0+  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
0-  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

           4        4-        3+         3        3-        2+         2  \
6+  0.001183  0.000000  0.000591  0.000591  0.001774  0.000000  0.000591   
6   0.001538  0.002308  0.000000  0.000000  0.001538  0.000000  0.000000   
6-  0.005095  0.004367  0.001456  0.000728  0.000728  0.000000  0.000000   
5+  0.036643  0.013002  0.007092  0.002364  0.005910  0.001182  0.000000   
5   0.068750  0.008824  0.006618  0.005147  0.003676  0.001471  0.000735   
5-  0.082898  0.018930  0.008486  0.005548  0.005222  0.002611  0.001305   
4+  0.166428  0.044324  0.022591  0.010580  0.011724  0.004289  0.002574   
4   0.333416  0.094661  0.092437  0.016807  0.016560  0.002966  0.003213   
4-  0.144644  0.289594  0.144034  0.033567  0.021666  0.009155  0.005798   
3+  0.068631  0.127569  0.246607  0.164017  0.052346  0.014734  0.012796   
3   0.042642  0.065003  0.132085  0.218929  0.162246  0.034841  0.016121   
3-  0.026602  0.045949  0.078597  0.107013  0.321644  0.052600  0.025393   
2+  0.020633  0.024759  0.050894  0.083906  0.126547  0.209078  0.052270   
2   0.012821  0.010684  0.032051  0.047009  0.072650  0.089744  0.198718   
2-  0.014327  0.011461  0.028653  0.048711  0.037249  0.097421  0.088825   
1+  0.004854  0.009709  0.014563  0.038835  0.067961  0.067961  0.063107   
1   0.000000  0.000000  0.017143  0.034286  0.051429  0.051429  0.045714   
1-  0.006579  0.006579  0.006579  0.006579  0.013158  0.026316  0.019737   
0+  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
0-  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          2-        1+         1        1-        0+         0        0-  \
6+  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6-  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5+  0.000000  0.000591  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000368  0.000368  0.000000   
5-  0.001632  0.000979  0.000000  0.000326  0.000000  0.000653  0.000000   
4+  0.001144  0.001144  0.000000  0.000286  0.000000  0.000000  0.000000   
4   0.002966  0.001977  0.000741  0.000247  0.000741  0.

## Syntetic data demo

### Step 1. Load & transform data

Below is some sythetic data generated from a dictionary of migration probabilities. You can change the probabilities of different data occurences (missing ratings, changed models, default rate) and see how the results are affected. Note that this example has some data features that do not exist in the sample data above. Missing ratings or models will be handled on both axes of the migration matrix. If this is not desired in the final table, data can be filtered before sending to the `migration_matrix` function.

In [76]:
# create test data
# category_list = ['6+', '5+', '4', '3-', '2-', 'U']
# 5 rating grades, migration is decribed in a dict-of-dicts

d_migration = {'6+': {'6+': 0.8, '5+': 0.08, '4': 0.05, '3-': 0.05, '2-': 0.0, 'U': 0.02},
               '5+': {'6+': 0.05, '5+': 0.7, '4': 0.18, '3-': 0.025, '2-': 0.025, 'U': 0.02},
               '4': {'6+': 0.05, '5+': 0.18, '4': 0.6, '3-': 0.1, '2-': 0.05, 'U': 0.02},
               '3-': {'6+': 0.0, '5+': 0.1, '4': 0.18, '3-': 0.5, '2-': 0.2, 'U': 0.02},
               '2-': {'6+': 0.05, '5+': 0.15, '4': 0.2, '3-': 0.18, '2-': 0.4, 'U': 0.02},
               'U': {'6+': 0.2, '5+': 0.18, '4': 0.2, '3-': 0.2, '2-': 0.2, 'U': 0.02}}

num_grades = len(d_migration)
num_act_per_grade = 10_000
total_act = num_grades * num_act_per_grade
dfr = pd.DataFrame.from_dict({'group': ['XT' + str(i).zfill(int(np.log10(total_act) + 2))
                                        for i in range(total_act)],
                              'rating_x': [np.random.choice(['6+', '5+', '4', '3-', '2-', 'U'])
                                           for i in range(total_act)]
                              })
dfr['FGRAD_MET'] = '03'
dfr['dflag'] = 0
dfr['rating_y'] = dfr['rating_x'].apply(lambda x: np.random.choice(a=list(d_migration[x].keys()),
                                                                   p=list(d_migration[x].values())))
df_1 = dfr[['group', 'rating_x', 'FGRAD_MET', 'dflag']].copy().rename(columns={'rating_x': 'Rating'})
df_2 = dfr[['group', 'rating_y', 'FGRAD_MET', 'dflag']].copy().rename(columns={'rating_y': 'Rating'})

# control for adding defaults, model changes, terminated accounts, missing ratings, and missing models
frac_total_defaults = 0.005
frac_total_deletes = 0.0002
frac_new_model = 0.0004
frac_miss_rating = 0.0003
frac_miss_model = 0.0001
random_seed = 23

# implement defaults in df1 and df2 at random
n_defaults = int(frac_total_defaults * total_act) + 1
print(f"Total number of defaults added to each data set: {n_defaults}")
np.random.seed(2*random_seed)
def_indices_1 = np.random.choice(df_1.index, n_defaults, replace=False)
df_1['dflag'] = 0
df_1.loc[def_indices_1, 'dflag'] = 1
# default flag in second data set is not used
#np.random.seed(3*random_seed)
#def_indices_2 = np.random.choice(df_2.index, n_defaults, replace=False)
#df_2['dflag'] = 0
#df_2.loc[def_indices_2, 'dflag'] = 1

# randomly delete rows to mimic customers joining and leaving between observation periods
n_deletes = int(frac_total_deletes * total_act) + 1
print(f"Total number of deleted accounts in each data set: {n_deletes}")
np.random.seed(4*random_seed)
drop_indices_1 = np.random.choice(df_1.index, n_deletes, replace=False)
#df_1 = df_1.drop(drop_indices_1)
np.random.seed(5*random_seed)
drop_indices_2 = np.random.choice(df_2.index, n_deletes, replace=False)
df_2 = df_2.drop(drop_indices_2)

# randomly assign customers to a different model in the second data set
n_other_model = int(frac_new_model * total_act) + 1
print(f"Total number of accounts with a new model in second data set: {n_other_model}")
np.random.seed(6*random_seed)
model_indices_2 = np.random.choice(df_2.index, n_other_model, replace=False)
df_2.loc[model_indices_2, 'FGRAD_MET'] = '06'

# randomly delete ratings, this is different from a specifically 'unrated' customer 'U' grade
n_miss_rating = int(frac_miss_rating * total_act) + 1
print(f"Total number of accounts missing a rating: {n_miss_rating}")
np.random.seed(7*random_seed)
miss_indices_1 =  np.random.choice(df_1.index, n_miss_rating, replace=False)
df_1.loc[miss_indices_1, 'Rating'] = np.nan
np.random.seed(8*random_seed)
miss_indices_2 =  np.random.choice(df_2.index, n_miss_rating, replace=False)
df_2.loc[miss_indices_2, 'Rating'] = np.nan

# randomly delete model
n_miss_model = int(frac_miss_model * total_act) + 1
print(f"Total number of accounts missing a model: {n_miss_model}")
np.random.seed(9*random_seed)
modmiss_indices_1 =  np.random.choice(df_1.index, n_miss_model, replace=False)
df_1.loc[modmiss_indices_1, 'FGRAD_MET'] = np.nan
np.random.seed(10*random_seed)
modmiss_indices_2 =  np.random.choice(df_2.index, n_miss_model, replace=False)
df_2.loc[modmiss_indices_2, 'FGRAD_MET'] = np.nan

Total number of defaults added to each data set: 301
Total number of deleted accounts in each data set: 13
Total number of accounts with a new model in second data set: 25
Total number of accounts missing a rating: 19
Total number of accounts missing a model: 7


### Step 2. Create the migration matrix

In [77]:
# create and test, using pandas.DataFrames
mm2 = migration_matrix(data_0=df_1, data_1=df_2, 
                       id_col='group', rating_col='Rating', 
                       model_col='FGRAD_MET', default_col='dflag')
print("The full migration matrix for the synthetic data:")
display(mm2)
df_mult = mm2.multiply(mm2.N, axis=0)
df_mult = df_mult.drop(labels='N', axis=1)
print("The actual numbers of customers in each cell:")
display(df_mult)
dff_sums = df_mult[[c for c in df_mult.columns if c in ['Defaults', 'Other model', 'Relationship terminated', 'Missing rating', 'Missing model']]].sum(axis=0)
print("Sum of 'to' columns:")
display(dff_sums)

The full migration matrix for the synthetic data:


6+        5+         4        3-        2-         U  \
6+              0.795019  0.078236  0.051421  0.049011  0.000000  0.019584   
5+              0.050605  0.703470  0.171817  0.023802  0.024902  0.019202   
4               0.049565  0.182972  0.597582  0.097732  0.047367  0.019286   
3-              0.000000  0.097541  0.174355  0.509348  0.195387  0.018695   
2-              0.050911  0.148078  0.201268  0.173138  0.400753  0.019513   
U               0.200139  0.176710  0.191303  0.201628  0.203117  0.020252   
Missing rating  0.263158  0.105263  0.315789  0.263158  0.052632  0.000000   
Missing model   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

                Defaults  Missing model  Other model  Missing rating  \
6+              0.005725       0.000000     0.000402        0.000402   
5+              0.004800       0.000000     0.000900        0.000400   
4               0.004397       0.000200     0.000200        0.000400   
3-              0.003759       0.000305     0.000203        0.000203   
2-              0.005151       0.000000     0.000594        0.000396   
U               0.006254       0.000199     0.000099        0.000000   
Missing rating  0.000000       0.000000     0.000000        0.000000   
Missing model   0.000000       0.000000     1.000000        0.000000   

                Relationship terminated        N  
6+                             0.000201   9957.0  
5+                             0.000100   9999.0  
4                              0.000300  10007.0  
3-                             0.000203   9842.0  
2-                             0.000198  10096.0  
U                              0.000298  10073.0  
Missing rating                 0.000000     19.0  
Missing model                  0.000000      7.0

The actual numbers of customers in each cell:


6+      5+       4      3-      2-      U  Defaults  \
6+              7916.0   779.0   512.0   488.0     0.0  195.0      57.0   
5+               506.0  7034.0  1718.0   238.0   249.0  192.0      48.0   
4                496.0  1831.0  5980.0   978.0   474.0  193.0      44.0   
3-                 0.0   960.0  1716.0  5013.0  1923.0  184.0      37.0   
2-               514.0  1495.0  2032.0  1748.0  4046.0  197.0      52.0   
U               2016.0  1780.0  1927.0  2031.0  2046.0  204.0      63.0   
Missing rating     5.0     2.0     6.0     5.0     1.0    0.0       0.0   
Missing model      0.0     0.0     0.0     0.0     0.0    0.0       0.0   

                Missing model  Other model  Missing rating  \
6+                        0.0          4.0             4.0   
5+                        0.0          9.0             4.0   
4                         2.0          2.0             4.0   
3-                        3.0          2.0             2.0   
2-                        0.0          6.0             4.0   
U                         2.0          1.0             0.0   
Missing rating            0.0          0.0             0.0   
Missing model             0.0          7.0             0.0   

                Relationship terminated  
6+                                  2.0  
5+                                  1.0  
4                                   3.0  
3-                                  2.0  
2-                                  2.0  
U                                   3.0  
Missing rating                      0.0  
Missing model                       0.0

Sum of 'to' columns:


Defaults                   301.0
Missing model                7.0
Other model                 31.0
Missing rating              18.0
Relationship terminated     13.0
dtype: float64